# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex, 
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 39 new papers today
          13 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/12 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2208.11128


extracting tarball to tmp_2208.11128...

 done.
Retrieving document from  https://arxiv.org/e-print/2208.11135


extracting tarball to tmp_2208.11135...

...

extracting tarball to tmp_2208.11683...

 done.
Retrieving document from  https://arxiv.org/e-print/2208.11686


/opt/hostedtoolcache/Python/3.9.13/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3398: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.13/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3398: LatexWarning: Found documentclass in tmp_2208.11683/main.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.13/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:292: LatexWarning: Latex injecting: 'author_list' from 'tmp_2208.11683/author_list.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))


extracting tarball to tmp_2208.11686...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2208.11360-b31b1b.svg)](https://arxiv.org/abs/arXiv:2208.11360) | **Building a GRAVITY+ Adaptive Optics Test Bench**  |
|| Gravity Plus Consortium, et al. -- incl., <mark>Frank Eisenhauer</mark>, <mark>Laura Kreidberg</mark> |
|*Appeared on*| *2022-08-25*|
|*Comments*| **|
|**Abstract**| We present the testbench aimed at integrating the GRAVITY+ adaptive optics GPAO. It consists of two independent elements, one reproducing the Coud{\'e} focus of the telescope, including the telescope deformable mirror mount (with its surface facing down), and one reproducing the Coud{\'e} room opto-mechanical environment, including a downwards-propagating beam, and the telescope mechanical interfaces in order to fit in the new GPAO wavefront sensor. We discuss in this paper the design of this bench and the solutions we adopted to keep the cost low, keep the design compact (allowing it to be fully contained in a 20 sqm clean room), and align the bench independently from the adaptive optics. We also discuss the features we have set in this bench. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2208.11128-b31b1b.svg)](https://arxiv.org/abs/arXiv:2208.11128) | **N-bearing complex organics toward high-mass protostars: Constant ratios  pointing to formation in similar pre-stellar conditions across a large mass  range**  |
|| P. Nazari, et al. -- incl., <mark>D. Langeroodi</mark> |
|*Appeared on*| *2022-08-25*|
|*Comments*| *49 pages, 46 figures, Accepted for publication in A&A*|
|**Abstract**| No statistical study of COMs toward a large sample of high-mass protostars with ALMA has been carried out so far. We aim to study six N-bearing species: CH$_3$CN, HNCO, NH$_2$CHO, C$_2$H$_5$CN, C$_2$H$_3$CN and CH$_3$NH$_2$ in a large sample of high-mass protostars. From the ALMAGAL survey, 37 of the most line-rich hot molecular cores are selected. Next, we fit their spectra and find column densities and excitation temperatures of the above N-bearing species, in addition to CH$_3$OH. We (tentatively) detect CH$_3$NH$_2$ in $\sim32%$ of the sources. We find three groups of species when comparing their excitation temperatures: hot (NH$_2$CHO; Tex > 250 K), warm (C$_2$H$_3$CN, HN$^{13}$CO and CH$_{3}^{13}$CN; 100 K < Tex < 250 K) and cold species (CH$_3$OH and CH$_3$NH$_2$; Tex < 100 K). This temperature segregation reflects the trend seen in their sublimation temperatures and validates the idea of onion-like structure of COMs around protostars. Moreover, the molecules studied here show constant column density ratios across low- and high-mass protostars with scatter less than a factor $\sim3$ around the mean. The constant column density ratios point to a common formation environment of COMs or their precursors, most likely in the pre-stellar ices. The scatter around the mean of the ratios, although small, varies depending on the species considered. This spread can either have a physical origin (source structure, line or dust optical depth) or a chemical one. Formamide is most prone to the physical effects as it is tracing the closest regions to the protostars, whereas such effects are small for other species. Assuming that all molecules form in the pre-stellar ices, the scatter variations could be explained by differences in lifetimes or physical conditions of the pre-stellar clouds. If the pre-stellar lifetimes are the main factor, they should be similar for low- and high-mass protostars. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2208.11135-b31b1b.svg)](https://arxiv.org/abs/arXiv:2208.11135) | **Energy wrinkles and phase-space folds of the last major merger**  |
|| Vasily Belokurov, et al. -- incl., <mark>Elliot Y. Davies</mark> |
|*Appeared on*| *2022-08-25*|
|*Comments*| *16 pages, 14 figures, submitted to MNRAS. Comments are welcome!*|
|**Abstract**| Relying on the dramatic increase in the number of stars with full 6D phase-space information provided by the Gaia Data Release 3, we discover unambiguous signatures of phase-mixing in the stellar halo around the Sun. We show that for the stars likely belonging to the last massive merger, the (v_r,r) distribution contains a series of long and thin chevron-like overdensities. These phase-space sub-structures are predicted to emerge following the dissolution of a satellite, when its tidal debris is given time to wind up, thin out and fold. Additionally, the observed energy and angular momentum (E, L_z) distribution appears more prograde at high energies, possibly revealing the original orbital angular momentum of the in-falling galaxy. The energy distribution of the debris is strongly asymmetric with a peak at low E -- which, we surmise, may be evidence of the dwarf's rapid sinking -- and riddled with wrinkles and bumps. If these small-scale energy inhomogeneities have been seeded during or immediately after the interaction with the Milky Way, and are not due to the spatial restriction of our study, then making use of the (v_r,r) chevrons to constrain the time of the merger becomes cumbersome. Nonetheless, we demonstrate that similar phase-space and (E,L_z) sub-structures are present in numerical simulations of galaxy interactions, both in bespoke N-body runs and in cosmological hydrodynamical zoom-in suites. The remnant traces of the progenitor's disruption and the signatures of the on-going phase-mixing discovered here will not only help to constrain the properties of our Galaxy's most important interaction, but also can be used as a novel tool to map out the Milky Way's current gravitational potential and its perturbations. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2208.11589-b31b1b.svg)](https://arxiv.org/abs/arXiv:2208.11589) | **The population of Galactic centre filaments III: candidate radio and  stellar sources**  |
|| F. Yusef-Zadeh, et al. -- incl., <mark>I. Heywood</mark> |
|*Appeared on*| *2022-08-25*|
|*Comments*| *63 pages, 3 Figures (multiple subfigures), MNRAS (in press)*|
|**Abstract**| Recent MeerKAT radio continuum observations of the Galactic center at 20 cm show a large population of nonthermal radio filaments (NRFs) in the inner few hundred pc of the Galaxy. We have selected a sample of 57 radios ources, mainly compact objects, in the MeerKAT mosaic image that appear to be associated with NRFs. The selected sources are about 4 times the number of radio point sources associated with filaments than would be expected by random chance. Furthermore, an apparent correlation between bright IR stars and NRFs is inferred from their similar latitude distributions, suggesting that they both co-exist within the same region. To examine if compact radio sources are related to compact IR sources, we have used archival 2MASS, and {\em Spitzer} data to make spectral energy distribution of individual stellar sources coincident or close to radio sources. We provide a catalogue of radio and IR sources for future detailed observations to investigate a potential 3-way physical association between NRFs, compact radio and IR stellar sources. This association is suggested by models in which NRFs are cometary tails produced by the interaction of a large-scale nuclear outflow with stellar wind bubbles in the Galactic center. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2208.11620-b31b1b.svg)](https://arxiv.org/abs/arXiv:2208.11620) | **A high angular resolution view of the PAH emission in Seyfert galaxies  using JWST/MRS data**  |
|| I. García-Bernete, et al. -- incl., <mark>A. Alonso-Herrero</mark>, <mark>C. Ramos Almeida</mark> |
|*Appeared on*| *2022-08-25*|
|*Comments*| *12 pages, 10 figures. Submitted to A&A*|
|**Abstract**| Polycyclic Aromatic Hydrocarbons (PAHs) are carbon-based molecules, which are ubiquitous in a variety of astrophysical objects and environments. In this work, we use JWST/MIRI MRS spectroscopy of 3 Seyferts to compare their nuclear PAH emission with that of star-forming regions. This study represents the first of its kind using sub-arcsecond angular resolution data of local luminous Seyferts (Lbol>10^44.89 erg/s) on a wide wavelength coverage (4.9-28.1 micron). We present an analysis of their nuclear PAH properties by comparing the observed ratios with PAH diagnostic model grids, derived from theoretical spectra. Our results show that a suite of PAH features is present in the innermost parts (~0.45 arcsec at 12 micron; in the inner ~142-245 pc) of luminous Seyfert galaxies. We find that the nuclear regions of AGN lie at different positions of the PAH diagnostic diagrams, whereas the SF regions are concentrated around the average values of SF galaxies. In particular, we find that the nuclear PAH emission mainly originates in neutral PAHs. In contrast, PAH emission originating in the SF regions favours ionised PAH grains. The observed PAH ratios in the nuclear region of AGN-dominated galaxy NGC 6552 indicate the presence of larger-sized PAH molecules compared with those of the SF regions. Therefore, our results provide evidence that the AGN have a significant impact on the ionization state (and probably the size) of the PAH grains on scales of ~142-245 pc. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2208.11683-b31b1b.svg)](https://arxiv.org/abs/arXiv:2208.11683) | **DESI Observations of the Andromeda Galaxy: Revealing the Immigration  History of our Nearest Neighbor**  |
|| Arjun Dey, et al. -- incl., <mark>J. Josephy-Zack</mark>, <mark>D. Lang</mark> |
|*Appeared on*| *2022-08-25*|
|*Comments*| *42 pages, 22 figures, 7 tables; submitted to Astrophysical Journal*|
|**Abstract**| We present DESI observations of the inner halo of M31, which reveal the kinematics of a recent merger - a galactic immigration event - in exquisite detail. Of the 11,416 sources studied in 3.75 hours of on-sky exposure time, 7,438 are M31 sources with well measured radial velocities. The observations reveal intricate coherent kinematic structure in the positions and velocities of individual stars: streams, wedges, and chevrons. While hints of coherent structures have been previously detected in M31, this is the first time they have been seen with such detail and clarity in a galaxy beyond the Milky Way. We find clear kinematic evidence for shell structures in the Giant Stellar Stream, the NE Shelf and Western Shelf regions. The kinematics are remarkably similar to the predictions of dynamical models constructed to explain the spatial morphology of the inner halo. The results are consistent with the interpretation that much of the substructure in the inner halo of M31 is produced by a single galactic immigration event 1 - 2 Gyr ago. Significant numbers of metal-rich stars are present in all of the detected substructures, suggesting that the immigrating galaxy had an extended star formation history. We also investigate the ability of the shells and Giant Stellar Stream to constrain the gravitational potential of M31, and estimate the mass within a projected radius of 125 kpc to be ${\rm log_{10}}\, M_{\rm NFW}(<125\,{\rm kpc})/M_\odot = 11.78_{-0.10}^{+0.13}$. The results herald a new era in our ability to study stars on a galactic scale and the immigration histories of galaxies. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2208.11686-b31b1b.svg)](https://arxiv.org/abs/arXiv:2208.11686) | **A roadmap to cosmological parameter analysis with third-order shear  statistics I: Modelling and validation**  |
|| <mark>Sven Heydenreich</mark>, Laila Linke, Pierre Burger, Peter Schneider |
|*Appeared on*| *2022-08-25*|
|*Comments*| **|
|**Abstract**| In this work, which is the first of a series to prepare a cosmological parameter analysis with third-order cosmic shear statistics, we model both the shear three-point correlation functions $\Gamma^{(i)}$ and the third-order aperture statistics $\langle\mathcal{M}_\mathrm{ap}^3\rangle$ from the BiHalofit bispectrum model and validate these statistics with a series of N-body simulations. We then investigate how to bin the shear three-point correlation functions to achieve an unbiased estimate for third-order aperture statistics in real data. Finally, we perform a cosmological parameter analysis on KiDS1000-like mock data with second- and third-order statistics. We recover all cosmological parameters with very little bias. Furthermore, we find that a joint analysis almost doubles the constraining power on $S_8$ and increases the figure-of-merit in the $\Omega_\mathrm{m}$-$\sigma_8$ plane by a factor of 5.9 with respect to an analysis with only second-order shear statistics. Our modelling pipeline is publicly available at https://github.com/sheydenreich/threepoint/releases/. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2208.11396-b31b1b.svg)](https://arxiv.org/abs/arXiv:2208.11396) | **Abundant ammonium hydrosulphide embedded in cometary dust grains**  |
|| K. Altwegg, et al. -- incl., <mark>A. Mahjoub</mark>, <mark>D. R. Müller</mark> |
|*Appeared on*| *2022-08-25*|
|*Comments*| *accepted by MNRAS*|
|**Abstract**| Ammonium hydrosulphide has long since been postulated to exist at least in certain layers of the giant planets. Its radiation products may be the reason for the red colour seen on Jupiter. Several ammonium salts, the products of NH3 and an acid, have previously been detected at comet 67P/Churyumov-Gerasimenko. The acid H2S is the fifth most abundant molecule in the coma of 67P followed by NH3. In order to look for the salt NH4+SH-, we analysed in situ measurements from the Rosetta/ROSINA Double Focusing Mass Spectrometer during the Rosetta mission. NH3 and H2S appear to be independent of each other when sublimating directly from the nucleus. However, we observe a strong correlation between the two species during dust impacts, clearly pointing to the salt. We find that NH4+SH- is by far the most abundant salt, more abundant in the dust impacts than even water. We also find all previously detected ammonium salts and for the first time ammonium fluoride. The amount of ammonia and acids balance each other, confirming that ammonia is mostly in the form of salt embedded into dust grains. Allotropes S2 and S3 are strongly enhanced in the impacts, while H2S2 and its fragment HS2 are not detected, which is most probably the result of radiolysis of NH4+SH-. This makes a prestellar origin of the salt likely. Our findings may explain the apparent depletion of nitrogen in comets and maybe help to solve the riddle of the missing sulphur in star forming regions. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2208.11360.md
    + _build/html/tmp_2208.11360/./Source.png
    + _build/html/tmp_2208.11360/./Bench_Schematics.jpg
    + _build/html/tmp_2208.11360/./WFS_fond_blanc2.jpg
    + _build/html/tmp_2208.11360/./G+_Implantation2.jpg
    + _build/html/tmp_2208.11360/./Phase_Mask.jpg
    + _build/html/tmp_2208.11360/./Alignment_Masks.jpg
    + _build/html/tmp_2208.11360/./DM_Mount.jpg
    + _build/html/tmp_2208.11360/./PSF_Jolie22.jpg
    + _build/html/tmp_2208.11360/./PSF_speckle.jpg
    + _build/html/tmp_2208.11360/./Pupil_Phasemask_crop.jpg


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\baselinestretch}{1.0}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\baselinestretch$}{1.0}$</div>



<div id="title">

# Building a GRAVITY+ Adaptive Optics Test Bench

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2208.11360-b31b1b.svg)](https://arxiv.org/abs/2208.11360)<mark>Appeared on: 2022-08-25</mark> - __

</div>
<div id="authors">

Gravity Plus Consortium, et al. -- incl., <mark><mark>Frank Eisenhauer</mark></mark>, <mark><mark>Laura Kreidberg</mark></mark>

</div>
<div id="abstract">

**Abstract:** We present the testbench aimed at integrating the GRAVITY+ adaptive optics GPAO. It consists of two independent elements, one reproducing the Coudé focus of the telescope, including the telescope deformable mirror mount (with its surface facing down), and one reproducing the Coudé room opto-mechanical environment, including a downwards-propagating beam, and the telescope mechanical interfaces in order to fit in the new GPAO wavefront sensor. We discuss in this paper the design of this bench and the solutions we adopted to keep the cost low, keep the design compact (allowing it to be fully contained in a 20 sqm clean room), and align the bench independently from the adaptive optics. We also discuss the features we have set in this bench.

</div>

<div id="div_fig1">

<img src="tmp_2208.11360/./Source.png" alt="Fig1.1" width="25%"/><img src="tmp_2208.11360/./Bench_Schematics.jpg" alt="Fig1.2" width="25%"/><img src="tmp_2208.11360/./WFS_fond_blanc2.jpg" alt="Fig1.3" width="25%"/><img src="tmp_2208.11360/./G+_Implantation2.jpg" alt="Fig1.4" width="25%"/>

**Figure 1. -** {\bf Top-Left:} CAD model of the testbench NGS light source and a sketch of the LGS light source. {\bf Top-Right:} GPAO test bench telescope simulator CAD view, with the Corrective Optics support structure on the top-left. Optical elements have all a 3-letter acronym recalling their functions (see text for details). {\bf Bottom-Left:} WFS support stucture CAD view.  {\bf Bottom-Right:} The GPAO testbench in its clean room. (*fig:schematics*)

</div>
<div id="div_fig2">

<img src="tmp_2208.11360/./Phase_Mask.jpg" alt="Fig3.1" width="33%"/><img src="tmp_2208.11360/./Alignment_Masks.jpg" alt="Fig3.2" width="33%"/><img src="tmp_2208.11360/./DM_Mount.jpg" alt="Fig3.3" width="33%"/>

**Figure 3. -** {\bf From left to right:} SILIOS phase plate installed on its rotating mount.
    3D-printed masks used for the alignment of the testbench. Installation of the Corrective Optics in Nice with its "dummy" flat mirror that replaces the Deformable Mirror for optical alignments. (*fig:phasemask*)

</div>
<div id="div_fig3">

<img src="tmp_2208.11360/./PSF_Jolie22.jpg" alt="Fig4.1" width="33%"/><img src="tmp_2208.11360/./PSF_speckle.jpg" alt="Fig4.2" width="33%"/><img src="tmp_2208.11360/./Pupil_Phasemask_crop.jpg" alt="Fig4.3" width="33%"/>

**Figure 4. -** {\bf Left:} GPAO testbench PSF core (hyperbolic arc-sinus scale), measured with the visible bench camera. {\bf Middle:} visible PSF with the phase mask in place (screen shot). {\bf Right:} Pupil vizualisation, showing the UT mask (in black) in front of the phase mask (giving an uniform grainy pattern). (*fig:psfpup*)

</div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

21  publications in the last 7 days.
	 _build/html/2208.11360.md
	 _build/html/2208.09335.md
	 _build/html/2208.08872.md
	 _build/html/2204.06393.md
	 _build/html/2204.03335.md
	 _build/html/2204.03253.md
	 _build/html/2204.02998.md
	 _build/html/2204.02109.md
	 _build/html/2204.02017.md
	 _build/html/2204.01824.md
	 _build/html/2204.01758.md
	 _build/html/2204.01245.md
	 _build/html/2204.00793.md
	 _build/html/2204.00342.md
	 _build/html/2203.16959.md
	 _build/html/2203.16856.md
	 _build/html/2203.16735.md
	 _build/html/2203.16734.md
	 _build/html/2203.16504.md
	 _build/html/2203.15822.md
	 _build/html/2203.15811.md


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

# Debugging papers